<a href="https://colab.research.google.com/github/Hongchenglong/colab/blob/main/CertifiableBayesianInference/FCN_Experiments/MNIST_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 装载Google硬盘
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys, os
from pathlib import Path
path = Path(os.getcwd())
sys.path.append(str(path.parent))
sys.path.append('/content/drive/MyDrive/CertifiableBayesianInference')
experiment = '/content/drive/MyDrive/ColabNotebooks/CertifiableBayesianInference/FCN_Experiments/'

In [10]:
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 0 --opt HMC --gpu 0 &
# python3 MNIST_runner.py --eps 0.11 --lam 0.25 --rob 2 --opt HMC --gpu 1 &

# dict = {'eps': 0.11, 'lam': 0.25, 'rob': 0, 'opt': 'HMC', 'gpu': '0'}
dict = {'eps': 0.11, 'lam': 0.25, 'rob': 2, 'opt': 'HMC', 'gpu': '1'}

# epsilon:  the strength of the adversary, 在测试点周围多大的区域里考虑它的鲁棒性
eps = dict['eps']
# lamada: the probability of seeing the clean data (lambda = 1.0 means no adversarial data considered, lambda = 0.0 means only adversarial data considered)
lam = dict['lam'] 
# robustness mode, 0:标准 1:IBP 2:PGD
rob = dict['rob'] 
# the variational inference method
optim = dict['opt']
gpu = dict['gpu']
os.environ['CUDA_VISIBLE_DEVICES'] = gpu

In [9]:
# 作者封装的BayesKeras模块
import BayesKeras
import BayesKeras.optimizers as optimizers

import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [5]:
# 划分训练集，测试集
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train/255.
X_test = X_test/255.
X_train = X_train.astype("float32").reshape(-1, 28*28)
X_test = X_test.astype("float32").reshape(-1, 28*28)

#X_train = X_train[0:10000]
#y_train = y_train[0:10000]

# 构建Sequential模型
model = Sequential()
# 构建Dense隐藏层并添加到模型中
# 添加具有512个神经元的Dense隐藏层，使用relu激活函数
model.add(Dense(512, activation="relu", input_shape=(None, None, 28*28)))
# 添加具有10个神经元的Dense隐藏层，使用softmax激活函数
model.add(Dense(10, activation="softmax"))

inf = 2
full_covar = False
if(optim == 'VOGN'):
    # was 0.25 for a bit
    inf = 2
    learning_rate = 0.35; decay=0.0
    opt = optimizers.VariationalOnlineGuassNewton()
elif(optim == 'BBB'):
    inf = 10
    learning_rate = 0.45; decay=0.0
    opt = optimizers.BayesByBackprop()
elif(optim == 'SWAG'):
    learning_rate = 0.01; decay=0.0
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SWAG-FC'):
    learning_rate = 0.01; decay=0.0; full_covar=True
    # 优化器.随机加权平均高斯
    opt = optimizers.StochasticWeightAveragingGaussian()
elif(optim == 'SGD'):
    learning_rate = 1.0; decay=0.0
    opt = optimizers.StochasticGradientDescent()
elif(optim == 'NA'):
    inf = 2
    learning_rate = 0.001; decay=0.0
    opt = optimizers.NoisyAdam()
elif(optim == 'ADAM'):
    learning_rate = 0.00001; decay=0.0
    opt = optimizers.Adam()
elif(optim == 'HMC'):
    learning_rate = 0.075; decay=0.0; inf=250
    linear_schedule = False
    # 哈密顿蒙特卡罗
    opt = optimizers.HamiltonianMonteCarlo()

# Compile the model to train with Bayesian inference
if(rob == 0):
    # 稀疏分类交叉熵
    loss = tf.keras.losses.SparseCategoricalCrossentropy()
elif(rob != 0):
    loss = BayesKeras.optimizers.losses.robust_crossentropy_loss

bayes_model = opt.compile(model, loss_fn=loss, epochs=20, learning_rate=learning_rate,
                          batch_size=128, linear_schedule=True,
                          decay=decay, robust_train=rob, c=inf,
                          burn_in=3, steps=25, b_steps=20, epsilon=eps, rob_lam=lam) #, preload="SGD_FCN_Posterior_1")

11501568/11490434 [==============================] - 0s 0us/step
This optimizer does not have a default compilation method. Please make sure to call the correct .compile method before use.
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
BayesKeras: Using implicit prior
(784, 512) 0.5646924393157821
(512, 10) 0.6987712429686843
BayesKeras: Detected robust training at compilation. Please ensure you have selected a robust-compatible loss
<class 'list'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
[<tf.Tensor: shape=(784, 512), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0.

In [6]:
# steps was 50
# Train the model on your data
bayes_model.train(X_train, y_train, X_test, y_test)

BayesKeras Warning: HMC is a memory hungry optimizer. 
         Given you system and parameters of this training run,
         we expect your system to need 0.002496 MB of available memory


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
Numerical Integration: 100%|██████████| 20/20 [02:44<00:00,  8.20s/it]


Current  U:  tf.Tensor(2.362415, shape=(), dtype=float32)
Proposed U:  tf.Tensor(0.656806, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.007950459, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.012483012, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(5.479843, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 1, loss: 3.130, acc: 0.612, val_loss: 0.606, val_acc: 0.878, rob: 0.000, (eps = 0.000000)
Still in Burn-In state of Markov chain.


Numerical Integration: 100%|██████████| 20/20 [02:47<00:00,  8.39s/it]


Current  U:  tf.Tensor(0.656806, shape=(), dtype=float32)
Proposed U:  tf.Tensor(0.54883325, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.008193829, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.008865946, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(1.1132689, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1]
Epoch 2, loss: 2.659, acc: 0.879, val_loss: 0.500, val_acc: 0.889, rob: 0.000, (eps = 0.036667)
Still in Burn-In state of Markov chain.


...

Current  U:  tf.Tensor(1.5925021, shape=(), dtype=float32)
Proposed U:  tf.Tensor(1.6639435, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.013712346, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.013774063, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.9309934, shape=(), dtype=float32)
REJECTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
Epoch 19, loss: 5.486, acc: 0.856, val_loss: 1.617, val_acc: 0.857, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [03:23<00:00,  8.15s/it]


Current  U:  tf.Tensor(1.5925021, shape=(), dtype=float32)
Proposed U:  tf.Tensor(1.698325, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.012494015, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.012518293, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.89956206, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1]
Epoch 20, loss: 5.556, acc: 0.854, val_loss: 1.598, val_acc: 0.860, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


Numerical Integration: 100%|██████████| 25/25 [03:23<00:00,  8.14s/it]


Current  U:  tf.Tensor(1.698325, shape=(), dtype=float32)
Proposed U:  tf.Tensor(1.747495, shape=(), dtype=float32)
Current  K:  tf.Tensor(0.014051455, shape=(), dtype=float32)
Proposed K:  tf.Tensor(0.014085925, shape=(), dtype=float32)
METROPOLIS CORRECTION CONSTANT:  tf.Tensor(0.95198643, shape=(), dtype=float32)
ACCEPTED
Debug info:
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1]
Epoch 21, loss: 5.682, acc: 0.851, val_loss: 1.630, val_acc: 0.856, rob: 0.000, (eps = 0.110000)
Out of Burn-In state. Generating samples from the chain


In [7]:
# Save your approxiate Bayesian posterior
bayes_model.save(experiment + "%s_FCN_Posterior_%s"%(optim, rob))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
